<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   wraps Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">

**Functools.wraps** ek decorator hai jo aapke wrapper function ko original function ke metadata (jaise ke docstring, function name, aur module information) copy karne mein madad karta hai.  
  
**Bina wraps ke, problem ye hoti hai:**  
- Jab aap koi decorator use karte hain, toh wrapper function original function ko wrap kar leta hai.  
- Is se original function ka **docstring, name, aur dusri attributes lost ho jati hain** aur wrapper function ka metadata show hota hai.  
- Yeh debugging, documentation, aur introspection mein problem create karta hai.

In [1]:
def my_decorator(func):
    
    def wrapper(*args, **kwargs):
        """This is my decorator"""
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper

In [2]:
@my_decorator
def greet(name):
    """This is greet function"""
    print(f"Hello, {name}!")

In [3]:
greet("Mubeen")

Hello, Mubeen!


<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
.__name__ current function ka name check karny k lye use kia gia hy

In [6]:
greet.__name__

'wrapper'

In [7]:
greet.__doc__

'This is my decorator'

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
        
#### Solution: **functools.wraps ka use**

In [3]:
from functools import wraps

In [18]:
def my_decorator(func):
    
    @wraps(func) # Yeh decorator original function ke metadata copy karega.
    
    def wrapper(*args, **kwargs):
        """This is my decorator"""
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper

In [24]:
@my_decorator
def greet(name):
    """This is greet function"""
    print(f"Hello, {name}!")

In [25]:
greet("Mubeen")

Hello, Mubeen!


In [26]:
greet.__name__

'greet'

In [27]:
greet.__doc__

'This is greet function'

<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   singledispatch Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
        
`functools.singledispatch` Python ka ek powerful tool hai jo aapko **generic functions** bananay ki sahulat deta hai. 

Iska matlab hai ke aap ek single function define kar sakte hain jo different types ke liye different behavior show kare—matlab type ke basis par function overloading. 

Yeh feature Python mein built-in function overloading ko simulate karta hai, kyunki Python mein traditionally overloading supported nahin hota.

---

#### **Why Use `singledispatch`? (Istemaal kyun karte hain?)**

1. **Code Organization aur Reusability:**
   - Aap ek generic function likh sakte hain jo multiple types handle kare.
   - Alag alag type-specific implementations ko ek hi function ke through organize kiya ja sakta hai.<br><br>

2. **Maintainability:**
   - Har type ke liye alag function likhne ki bajaye, aap base function define karke usko extend kar sakte hain.
   - Code ko modular aur asaan maintain karna hota hai.<br><br>

3. **Polymorphism:**
   - Function behavior ko type ke hisaab se change kar sakte hain bina function ke naam badle.


In [55]:
from functools import singledispatch

In [44]:
@singledispatch
def process_data(data):
    return f"Cannot process data of type {type(data)}"

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for Int

In [45]:
@process_data.register(int)
def _(data):
    print(f"Processing integer: {data}")
    return data * 2

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for string

In [47]:
@process_data.register(str)
def _(data):
    print(f"Processing string: {data}")
    return data.upper()

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for list

In [48]:
@process_data.register(list)
def _(data):
    print(f"Processing list with {len(data)} elements.")
    return [x * 2 for x in data]

In [49]:
process_data(10) # int

Processing integer: 10


20

In [50]:
process_data("mubeen") # string

Processing string: mubeen


'MUBEEN'

In [51]:
process_data([1,2,3]) # list

Processing list with 3 elements.


[2, 4, 6]

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> Example 2

In [58]:
import json

@singledispatch
def serialize(obj):
    return f"Serialization not supported for type {type(obj)}"

In [59]:
@serialize.register(dict)
def _(obj: dict):
    print("Serializing dictionary")
    return json.dumps(obj)

In [60]:
@serialize.register(list)
def _(obj: list):
    print("Serializing list")
    return json.dumps(obj)

In [61]:
@serialize.register(int)
def _(obj: int):
    print("Serializing integer")
    return str(obj)

In [63]:
# Custom class ke liye serializer.
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
@serialize.register(Person)
def _(obj: Person):
    print("Serializing Person object")
    return json.dumps({"name": obj.name, "age": obj.age})

In [65]:
serialize([1, 2, 3, 4])

Serializing list


'[1, 2, 3, 4]'

In [ ]:
serialize({"key": "value", "num": 42})

In [66]:
serialize(123)

Serializing integer


'123'

In [67]:
serialize(Person("Ali", 30))

Serializing Person object


'{"name": "Ali", "age": 30}'

<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   lru_cache Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> 
        
`functools.lru_cache` ek decorator hai jo function ke results ko cache (yaani temporarily store) kar leta hai.

Is se agar aap same arguments ke saath function ko dobara call karte hain, to function dobara calculation nahi karta, balkay cached result return kar deta hai. 
        
Yeh feature khas taur pe un functions ke liye bohot useful hai jin ki calculations expensive  hoti hain aur jinhen bar bar same input ke saath call kiya jata hai.

### **Key Points:**

- **LRU ka matlab hai "Least Recently Used":**  
  Yeh cache mechanism aisa hai ke agar cache full ho jaye (maxsize set ho), to sabse pehle un entries ko remove karta hai jo sab se kam use hui hain.
  
- **Performance Improvement:**  
  Agar koi function expensive calculation karta hai (jaise recursive functions, database queries, ya heavy computation), to caching se execution time significantly reduce ho sakta hai.
  
- **Usage:**  
  `@functools.lru_cache(maxsize=None)` se aap cache ka size specify kar sakte hain. Agar `maxsize` ko None set kar dein, to cache size unlimited ho jayegi.


In [104]:
from functools import lru_cache

In [182]:

@lru_cache(maxsize=4)
def fetch_data_from_api(user_id):
    print(f"Fetching data for user: {user_id} from API...")
    time.sleep(2)
    return f"Data for {user_id}"


In [183]:
print(fetch_data_from_api("user1"))  

Fetching data for user: user1 from API...
Data for user1


In [184]:
print(fetch_data_from_api("user2"))

Fetching data for user: user2 from API...
Data for user2


In [185]:
print(fetch_data_from_api("user3"))

Fetching data for user: user3 from API...
Data for user3


In [186]:
print(fetch_data_from_api("user1"))

Data for user1


<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> 
        
- **Function Definition:**  
  `fetch_data_from_api(user_id)` ek function hai jo simulate karta hai ke API se data fetch kar raha hai. Ismein 2 second ka delay use hota hai, taake expensive computation ya network delay ko dikhaya ja sake.

- **Decorator `@functools.lru_cache(maxsize=4)`:**  
  Is decorator ka matlab hai ke function ke results ko cache kar liya jayega. Agar same `user_id` ke sath function dobara call hota hai, to expensive calculation (ya delay) avoid ho jata hai aur direct cached result return hota hai.

- **Function Calls:**  
  - Jab pehli baar `fetch_data_from_api("user1")` call hota hai, to API call simulate hoti hai aur delay hota hai.  
  - Dobara agar "user1" call kiya jaye to result cached hota hai aur delay nahi hoti.  
  - Dusre user ids ("user2", "user3") ke liye naya computation hota hai aur unke results cache ho jate hain.

- **Real-World Use Case:**  
  Web applications ya APIs mein, agar bar bar same parameters ke sath heavy computation ya network request perform ho rahi ho, to `lru_cache` se response time improve hota hai aur server pe load kam hota hai.

---

Is tarah ka example aapko dikhata hai ke kaise `functools.lru_cache` se performance optimize kiya ja sakta hai.